In [1]:
!pip install langchain==0.3.0 langchain-openai==0.2.0 langgraph==0.2.22 httpx==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB

In [3]:
import operator
from typing import Annotated, Any, Optional

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from pydantic import BaseModel, Field

import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"


# ペルソナを表すデータモデル
class Persona(BaseModel):
    name: str = Field(..., description="ペルソナの名前")
    background: str = Field(..., description="ペルソナの背景情報")


# ペルソナのリストを表すデータモデル
class Personas(BaseModel):
    personas: list[Persona] = Field(
        default_factory=list, description="ペルソナのリスト"
    )


# インタビュー内容を表すデータモデル
class Interview(BaseModel):
    persona: Persona = Field(..., description="インタビュー対象のペルソナ")
    question: str = Field(..., description="インタビューでの質問")
    answer: str = Field(..., description="インタビューでの回答")


# インタビュー結果のリストを表すデータモデル
class InterviewResult(BaseModel):
    interviews: list[Interview] = Field(
        default_factory=list, description="インタビュー結果のリスト"
    )


# エージェントの内部状態を表すモデル
class InterviewState(BaseModel):
    user_request: str = Field(..., description="ユーザーからのリクエスト")
    personas: Annotated[list[Persona], operator.add] = Field(
        default_factory=list, description="生成されたペルソナのリスト"
    )
    interviews: Annotated[list[Interview], operator.add] = Field(
        default_factory=list, description="実施されたインタビューのリスト"
    )
    seo_strategy_doc: str = Field(default="", description="生成されたSEO戦略ドキュメント")
    iteration: int = Field(
        default=0, description="ペルソナ生成とインタビューの反復回数"
    )


# ペルソナを生成するクラス（SEOに興味のあるユーザーを想定）
class PersonaGenerator:
    def __init__(self, llm: ChatOpenAI, k: int = 5):
        self.llm = llm.with_structured_output(Personas)
        self.k = k

    def run(self, user_request: str) -> Personas:
        # --- SEO向けに変更したプロンプト ここから ---

        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたは依頼元が提供するサービスに関心を持ち得る潜在顧客のペルソナ作成に特化した専門家です。"
                    "それぞれのペルソナが具体的にどんな悩みを抱え、どんな情報を求めているかを明確に示してください。"
                ),
                (
                    "human",
                    (
                        f"以下の内容で、{self.k}人の多様なペルソナを生成してください。\n\n"
                        "【サービスの概要】\n{user_request}\n\n"
                        "各ペルソナには以下を含めてください:\n"
                        "- 名前\n"
                        "- 年齢、性別、職業\n"
                        "- どんな課題・悩みを抱えているか\n"
                        "- あなたのサービスを見つける可能性が高い情報源（検索エンジン、SNS、知人の紹介など）\n"
                        "- どんな情報（価格、口コミ、機能比較など）を特に重視するか\n"
                        "- どんなキーワードで検索しそうか（推定でOK）\n"
                        "具体的でリアルに想像しやすい設定をお願いします。"
                    ),
                ),
            ]
        )
        # --- SEO向けに変更したプロンプト ここまで ---
        chain = prompt | self.llm
        return chain.invoke({"user_request": user_request})


# インタビューを実施するクラス（SEOにおける悩みやニーズを深堀りする）
class InterviewConductor:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, user_request: str, personas: list[Persona]) -> InterviewResult:
        questions = self._generate_questions(user_request, personas)
        answers = self._generate_answers(personas, questions)
        interviews = self._create_interviews(personas, questions, answers)
        return InterviewResult(interviews=interviews)

    def _generate_questions(
        self, user_request: str, personas: list[Persona]
    ) -> list[str]:
        # --- SEO向けの質問生成用プロンプト ---
        question_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはインタビュアーです。潜在顧客の悩みや課題を深掘りし、"
                    "どのようにサービスを見つけ、選び、利用しようとしているのかを明らかにする質問を考えるプロです。"
                ),
                (
                    "human",
                    (
                        "以下の読者ペルソナが、あなたのサービスに対して抱えている悩みや、"
                        "検索やSNSを含む情報収集の実態を率直に話せるようなオープンな質問を作成してください。\n\n"
                        "【サービスの概要】\n{user_request}\n\n"
                        "【読者ペルソナ】\n{persona_name} - {persona_background}\n\n"
                        "質問のポイント:\n"
                        "- 現在の悩みや目的を具体的に引き出せる\n"
                        "- どこで情報収集しているかを明確にできる\n"
                        "- 何を重視して比較検討しているのかを知る\n"
                        "シンプルだが、回答者が深く考えられるようにしてください。"
                    ),
                ),
            ]
        )
        question_chain = question_prompt | self.llm | StrOutputParser()
        question_queries = [
            {
                "user_request": user_request,
                "persona_name": persona.name,
                "persona_background": persona.background,
            }
            for persona in personas
        ]
        return question_chain.batch(question_queries)

    def _generate_answers(
        self, personas: list[Persona], questions: list[str]
    ) -> list[str]:
        # --- SEO向けの回答生成用プロンプト ---

        answer_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたは以下のペルソナです。サービスに関して抱えている具体的な悩み、"
                    "どのように情報を探すのか、何を基準に選ぶのかなどを正直に答えてください。"
                ),
                (
                    "human",
                    (
                        "ペルソナ: {persona_name} - {persona_background}\n\n"
                        "質問: {question}\n\n"
                        "回答のポイント:\n"
                        "- どのような情報収集プロセスを踏んでいるか\n"
                        "- 検索エンジンを使う場合、どんなキーワードを想定しているか\n"
                        "- 何を重視しているか（価格、口コミ、評判、機能、サポート等）\n"
                        "- 具体的な利用シーンや期待している効果\n"
                    ),
                ),
            ]
        )

        answer_chain = answer_prompt | self.llm | StrOutputParser()
        answer_queries = [
            {
                "persona_name": persona.name,
                "persona_background": persona.background,
                "question": question,
            }
            for persona, question in zip(personas, questions)
        ]
        return answer_chain.batch(answer_queries)

    def _create_interviews(
        self, personas: list[Persona], questions: list[str], answers: list[str]
    ) -> list[Interview]:
        return [
            Interview(persona=persona, question=q, answer=a)
            for persona, q, a in zip(personas, questions, answers)
        ]


# SEO戦略ドキュメントを生成するクラス
class SEOStrategyDocumentGenerator:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, user_request: str, interviews: list[Interview]) -> str:
        # --- SEO戦略立案のためのプロンプト ---
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    (
                        "あなたはSEO戦略を立案するプロフェッショナルです。"
                        "以下の情報に基づいて、具体的で効果的なSEO戦略を提案してください。"
                        "キーワード調査やコンテンツの最適化、リンクビルディング、"
                        "技術的SEO（サイト速度・モバイル対応など）、競合分析など、"
                        "幅広い視点からアドバイスを行ってください。"
                    ),
                ),
                (
                    "human",
                    (
                        "以下のWebサイト概要と複数の読者ペルソナ（インタビュー結果）に基づいて、"
                        "実践的なSEO戦略ドキュメントを作成してください。"
                        "具体的なアクションプラン、優先度、ツールの活用方法、"
                        "必要に応じたリソースやコストの見積もりなども含めてください。\n\n"
                        "【Webサイトの概要】\n{user_request}\n\n"
                        "【インタビュー結果】\n{interview_results}\n\n"
                        "最低限、以下の項目を盛り込んでください:\n"
                        "1. SEO施策の目的（どんなKPIを達成したいか）\n"
                        "2. ターゲット層と主要キーワード\n"
                        "3. 現状の課題（技術的SEO、コンテンツ、バックリンクなど）\n"
                        "4. 施策の優先度とロードマップ（短期・中期・長期）\n"
                        "5. キーワード戦略やコンテンツ最適化の方針\n"
                        "6. リンクビルディングの戦略\n"
                        "7. 競合サイトの分析ポイント\n"
                        "8. 必要なツールやリソース、運用コストの目安\n"
                        "9. モニタリングと改善サイクル\n\n"
                        "以上を踏まえ、プロが実践できるレベルの詳細な戦略ドキュメントを作成してください。"
                    ),
                ),
            ]
        )
        chain = prompt | self.llm | StrOutputParser()

        # インタビュー結果をテキスト形式にまとめる
        interview_results_text = "\n".join(
            f"ペルソナ: {i.persona.name} - {i.persona.background}\n"
            f"質問: {i.question}\n回答: {i.answer}\n"
            for i in interviews
        )

        return chain.invoke(
            {
                "user_request": user_request,
                "interview_results": interview_results_text,
            }
        )


# 「SEO戦略立案AIエージェント」のクラス
class SEOAgent:
    def __init__(self, llm: ChatOpenAI, k: Optional[int] = None):
        self.persona_generator = PersonaGenerator(llm=llm, k=k)
        self.interview_conductor = InterviewConductor(llm=llm)
        self.seo_strategy_generator = SEOStrategyDocumentGenerator(llm=llm)
        self.graph = self._create_graph()

    def _create_graph(self) -> StateGraph:
        workflow = StateGraph(InterviewState)

        # ノードを追加
        workflow.add_node("generate_personas", self._generate_personas)
        workflow.add_node("conduct_interviews", self._conduct_interviews)
        workflow.add_node("generate_strategy", self._generate_strategy)

        # エントリーポイント
        workflow.set_entry_point("generate_personas")

        # 遷移設定
        workflow.add_edge("generate_personas", "conduct_interviews")
        workflow.add_edge("conduct_interviews", "generate_strategy")
        workflow.add_edge("generate_strategy", END)

        return workflow.compile()

    def _generate_personas(self, state: InterviewState) -> dict[str, Any]:
        new_personas = self.persona_generator.run(state.user_request)
        return {
            "personas": new_personas.personas,
            "iteration": state.iteration + 1,
        }

    def _conduct_interviews(self, state: InterviewState) -> dict[str, Any]:
        # ペルソナ数が多い場合、最後の5人のみに制限
        new_personas = state.personas[-5:]
        new_interviews = self.interview_conductor.run(
            state.user_request, new_personas
        )
        return {"interviews": new_interviews.interviews}

    def _generate_strategy(self, state: InterviewState) -> dict[str, Any]:
        seo_strategy_doc = self.seo_strategy_generator.run(
            state.user_request, state.interviews
        )
        return {"seo_strategy_doc": seo_strategy_doc}

    def run(self, user_request: str) -> str:
        initial_state = InterviewState(user_request=user_request)
        final_state = self.graph.invoke(initial_state)
        return final_state["seo_strategy_doc"]


def main():
    user_request = input("SEO戦略を立案したいWebサイトの概要や目的を入力してください: ")
    k = 3  # 生成するペルソナの人数（必要に応じて変更可能）

    # モデル名は利用できるものに合わせて変更してください
    llm = ChatOpenAI(model_name="gpt-4o-2024-11-20", temperature=0.0)
    agent = SEOAgent(llm=llm, k=k)
    final_output = agent.run(user_request=user_request)

    print(final_output)


if __name__ == "__main__":
    main()


SEO戦略を立案したいWebサイトの概要や目的を入力してください: 不動産業界向けの財務モデリング、Excelテンプレートの標準化・自動化、トレーニングなどの提供
# SEO戦略ドキュメント: 不動産業界向け財務モデリング・Excelテンプレート提供サイト

---

## 1. **SEO施策の目的**
### **目的**
- **リード獲得**: 不動産業界の財務部門や投資家、コンサルタントをターゲットに、問い合わせや資料ダウンロード、トライアル申し込みを増加させる。
- **トラフィック増加**: 月間オーガニックトラフィックを現状の2倍に増加。
- **コンバージョン率向上**: サイト訪問者のうち、問い合わせや資料請求に至るコンバージョン率を5%以上に向上。

### **KPI**
- 月間オーガニックトラフィック: **+100%増加**（例: 5,000→10,000セッション）
- リード獲得数: **月間50件以上**
- コンバージョン率: **5%以上**
- 主要キーワードでの検索順位: **上位3位以内を10個以上達成**

---

## 2. **ターゲット層と主要キーワード**
### **ターゲット層**
1. **田中一郎（不動産会社の財務部長）**
   - ニーズ: 財務モデリングの効率化、エラー削減、標準化
   - 情報収集方法: Google検索、業界セミナー、LinkedIn
   - 重視ポイント: 導入のしやすさ、既存Excelとの連携、コストパフォーマンス

2. **佐藤美咲（不動産投資家）**
   - ニーズ: 初心者でも使いやすい財務分析ツール、学習リソース
   - 情報収集方法: Google検索、YouTube、SNS
   - 重視ポイント: 価格、使いやすさ、初心者向けサポート

3. **山本健太（不動産コンサルタント）**
   - ニーズ: クライアント向けレポート作成の効率化、データの自動化
   - 情報収集方法: 業界セミナー、Google検索、LinkedIn
   - 重視ポイント: カスタマイズ性、サポート体制、実績

### **主要キーワード**
#### **短期（競合性低・ニッチ）**
- 「不動産 財務モデリング テンプレート」
- 「Excel 自動化 不動産」

# English Version